In [81]:
import torch
import torch.nn as nn
import numpy as np
from torch.autograd import Variable
from torchsummary import summary

In [82]:
#Tensor->（multidimensional array）

#d input nodes k hidden layer
K = 10
d = 10
batch_size = 1

X = Variable(torch.rand(batch_size,d),requires_grad=True)
y = torch.tensor([[torch.sum(X**2)/d]])

In [219]:
class NN1(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        global z,z_relu
        z = []
        z_relu = []
        super(NN1, self).__init__()
        self.layer = nn.Sequential()
        self.layer.add_module("h_1", nn.Linear(input_size, hidden_size))
        
        z.append(self.layer(X))
        z_relu.append(self.layer(X))
        for i in range(K-1):
             index = str(i+2)
             self.layer.add_module("h_"+index, nn.Linear(hidden_size, hidden_size))
             z.append(self.layer(X)) 
             self.layer.add_module("h_"+index+"_relu"+index,nn.ReLU())
             z_relu.append(self.layer(X))
                
        self.layer.add_module("h_last", nn.Linear(hidden_size, num_classes))
        z.append(self.layer(X))
        #self.layer.add_module("h_1ast_relu"+index,nn.ReLU())
        z_relu.append(self.layer(X))
        #print(len(self.layer))

    
    def forward(self, X):
        y = self.layer(X)
        return y
        

In [220]:
model = NN1(d,2*d+1,1)

In [85]:
y_pred = model(X)
loss = (y_pred - y)**2
loss.item()


0.0853477194905281

In [86]:
y_pred

tensor([[-0.12243]], grad_fn=<AddmmBackward>)

In [203]:
summary(model,(1,10))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 21]             231
            Linear-2                [-1, 1, 21]             462
              ReLU-3                [-1, 1, 21]               0
            Linear-4                [-1, 1, 21]             462
              ReLU-5                [-1, 1, 21]               0
            Linear-6                [-1, 1, 21]             462
              ReLU-7                [-1, 1, 21]               0
            Linear-8                [-1, 1, 21]             462
              ReLU-9                [-1, 1, 21]               0
           Linear-10                [-1, 1, 21]             462
             ReLU-11                [-1, 1, 21]               0
           Linear-12                [-1, 1, 21]             462
             ReLU-13                [-1, 1, 21]               0
           Linear-14                [-1

In [87]:
loss.backward()

--------------------

In [88]:
for param in model.named_parameters():
    print(param)

('layer.h_1.weight', Parameter containing:
tensor([[-0.06029,  0.21235, -0.18432, -0.21710,  0.01421, -0.06148,  0.20559,
          0.22207,  0.09164,  0.08526],
        [-0.26722, -0.15193,  0.00098, -0.18134, -0.14750, -0.13912,  0.25548,
          0.00385,  0.08259,  0.12665],
        [-0.12491, -0.06641, -0.17878, -0.17546,  0.12067,  0.24411, -0.10488,
          0.00510,  0.04441, -0.31352],
        [ 0.06282, -0.02817, -0.00139, -0.23598, -0.07781,  0.31447, -0.26170,
          0.09480,  0.16591,  0.09165],
        [ 0.13492, -0.25647,  0.16043,  0.00968,  0.00416,  0.07395,  0.08891,
         -0.20451, -0.12341, -0.09167],
        [ 0.04811, -0.05418, -0.29896, -0.14108, -0.26002, -0.26773,  0.20185,
         -0.25896, -0.27707, -0.07152],
        [ 0.01096, -0.26877,  0.07941, -0.02627, -0.12407,  0.09232, -0.28315,
         -0.08008,  0.27515, -0.09775],
        [ 0.03848,  0.10699, -0.06084,  0.16475,  0.01441,  0.13092, -0.04734,
          0.23502, -0.06237, -0.27383],
     

In [89]:
#save auto w,b
auto_w =[]
auto_b =[]
i = 0

for param in model.parameters():
    torch.set_printoptions(precision=5)
    if (i%2==0):
        auto_w.append(param.grad)
    if (i%2==1):
        auto_b.append(param.grad)
    i+=1

In [91]:
for i in range(len(auto_w)):
    auto_w[i]= np.round(np.array(auto_w[i]).astype(np.double),5)
    auto_b[i]= np.round(np.array(auto_b[i]).astype(np.double),5)

-----------------------

In [92]:
#手动grad share the same weight and bias with auto_forward
parameters = list(model.parameters())

In [97]:
#manual forward
def feedforward(X,parameters):
    global z_manual,z_relu_manual
    z_manual=[]
    z_relu_manual=[]
    z_manual.append(torch.from_numpy(np.dot(X.detach().numpy(),(parameters[0].detach().numpy().T))+parameters[1].detach().numpy()))
    z_relu_manual.append(z_manual[0])
    
    for i in range(0,10):
        z_manual.append(torch.from_numpy(np.dot(z_relu_manual[i],(parameters[(i+1)*2].detach().numpy().T))+parameters[(i+1)*2+1].detach().numpy()))
        z_relu_manual.append(ReLu(z_manual[i+1]))
    z_relu_manual[-1]=z_manual[-1]
    return z_relu_manual[-1]
y_pred = feedforward(X,parameters)

In [153]:
#求对z的的倒数
# dL/dv5
d_z = []
d_w = []
torch.set_printoptions(precision=5)
grad_y_pred = 2.0 * (y_pred - y) #[1,1]
grad_z_y = grad_y_pred.detach().numpy()#*ReLu_d(y_pred)#dL/dz5,5)
grad_w_y = grad_z_y*z_relu_manual[-2]#.detach().numpy()
d_z.append(grad_z_y)
d_w.append(grad_w_y)
#from the last hidden layer
for i in range (9):#i=1,2,3,4,5.... p:-4,-6,-8
    grad_z_i = np.dot(d_z[i],ReLu_d(z_manual[-i-2])*parameters[-(i+1)*2].detach().numpy())
    d_z.append(grad_z_i)
    grad_w_i = np.dot(grad_z_i.T,z_relu_manual[-3-i])#.detach().numpy() )#-4,-5,-6
    d_w.append(grad_w_i)
    
#first layer
grad_z_1 = np.dot(d_z[9],ReLu_d(z_manual[-11])*parameters[-20].detach().numpy())
d_z.append(grad_z_1)
grad_w_1 = np.dot(grad_z_1.T,X.detach().numpy())
d_w.append(grad_w_1)


-------------

In [14]:
#激活函数的倒数
def ReLu_d(x):
    return np.where(x < 0, 0, 1)
def ReLu(x):
    return np.where(x < 0, 0, x)

In [154]:
for i in range(len(d_w)):
    d_w[i]= np.round(np.array(d_w[i]).astype(np.double),5)
    d_z[i]= np.round(np.array(d_z[i]).astype(np.double),5)

In [155]:
d_w.reverse()

In [156]:
d_z.reverse()

In [222]:
#chaeck value difference between autograd and mygrad

In [211]:
diff_w=0
for i in range(len(d_w)):
    diff_w = diff_w+ np.absolute(sum(sum(d_w[i]-auto_w[i])))

In [216]:
diff_w

3.999999999999924e-05

In [213]:
diff_b=0
for i in range(len(d_z)):
    diff_b = diff_b+ np.absolute(sum(sum(d_z[i]-auto_b[i])))
diff_b

1e-05

In [214]:
with open('torch_autograd.dat', 'w') as f:
    f.write('autograd_w' + '\n')
    i = 1
    for w in auto_w:
        f.write('weight_' +str(i)+ '\n')
        f.write(str(w)+'\n')
        i+=1
    f.write('autograd_b' + '\n')
    i = 1
    for b in auto_b:
        f.write('bias_' +str(i)+ '\n')
        f.write(str(b)+'\r\n')
        i+=1
    

In [215]:
with open('my_autograd.dat', 'w') as f:
    f.write('my_w' + '\n')
    i = 1
    for w in d_w:
        f.write('weight_' +str(i)+ '\n')
        f.write(str(w)+'\n')
        i+=1
    f.write('my_b' + '\n')
    i = 1
    for b in d_z:
        f.write('bias_' +str(i)+ '\n')
        f.write(str(b)+'\r\n')
        i+=1
    